In [80]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
print("Current GPU:", torch.cuda.current_device())
print("GPU Name:", torch.cuda.get_device_name(0))

CUDA Available: True
GPU Count: 1
Current GPU: 0
GPU Name: NVIDIA GeForce GTX 1070


In [81]:
# %%
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap, TyreRaceData, Team, DriverTeamSession, TeamCircuitStats

# Initialize database session
db_engine, db_session = init_db()

# Fetch a specific race (e.g., 2023 Bahrain GP)
race_weekend = db_session.query(RacingWeekend).filter_by(year=2023, round=1).first()
race_session = db_session.query(Session).filter_by(weekend_id=race_weekend.racing_weekend_id, session_type='Race').first()

# Get total laps in the race
total_laps = db_session.query(func.max(Lap.lap_num)).filter_by(session_id=race_session.session_id).scalar()

# Fetch driver's first lap to determine starting tyre
driver_id = 12  # Example driver
first_lap = db_session.query(Lap).filter_by(session_id=race_session.session_id, driver_id=driver_id).order_by(Lap.lap_num).first()
initial_tyre = first_lap.tyre

# Fetch the last 20 races the driver participated in
last_20_race_ids = (
	db_session.query(TyreRaceData.race_id)
	.filter_by(driver_id=driver_id)
	.order_by(TyreRaceData.race_id.desc())  # Assuming race_id is incremental
	.limit(20)
	.subquery()
)

# Fetch tyre degradation parameters for the last 20 races and average them
tyre_data = (
	db_session.query(
		TyreRaceData.tyre_type,
		func.avg(TyreRaceData.a).label("avg_a"),
		func.avg(TyreRaceData.b).label("avg_b"),
		func.avg(TyreRaceData.c).label("avg_c"),
	)
	.filter(TyreRaceData.driver_id == driver_id)
	.filter(TyreRaceData.race_id.in_(last_20_race_ids))
	.group_by(TyreRaceData.tyre_type)
	.all()
)

# Store averaged values in dictionary
tyre_params = {
	td.tyre_type: {'a': td.avg_a, 'b': td.avg_b, 'c': td.avg_c}
	for td in tyre_data
}

print(tyre_params)

# Fetch team's pit time at the circuit
dts = db_session.query(DriverTeamSession).filter_by(session_id=race_session.session_id, driver_id=driver_id).first()
team_stats = db_session.query(TeamCircuitStats).filter_by(circuit_id=race_weekend.circut_id, team_id=dts.team_id).first()
pit_time = team_stats.pit_time

# Baseline laptime

team_stats = db_session.query(TeamCircuitStats).filter_by(
		circuit_id=race_weekend.circut_id,
		team_id=dts.team_id
	).first()
percent_diff = team_stats.quali_to_race_percent_diff  # Quali-to-race % difference

quali_session = db_session.query(Session).filter_by(
	weekend_id=race_weekend.racing_weekend_id,
	session_type='Qualifying'
).first()

if not quali_session:
	raise ValueError("No qualifying session found for this race weekend.")
quali_session_id = quali_session.session_id

fastest_lap = db_session.query(Lap).filter(
	Lap.session_id == quali_session_id,
	Lap.driver_id == 12
).order_by(Lap.lap_time.asc()).first()

if not fastest_lap:
	raise ValueError("No qualifying laps found for driver 12.")

print(f"Lap Number: {fastest_lap.lap_num}")
print(f"Lap Time: {fastest_lap.lap_time} seconds")
baseline_lap = (fastest_lap.lap_time) + (1 * percent_diff)
print(f"Lap Time: {baseline_lap} seconds")


{1: {'a': 0.014524335031135222, 'b': -0.259793656959313, 'c': 1.862598802478543}, 2: {'a': 0.003404626754128564, 'b': -0.0015711251891984146, 'c': 0.8881846067091267}, 3: {'a': 0.001548857741135055, 'b': -0.010664659121802356, 'c': 1.1093883954908919}, 4: {'a': 0.003961848445711819, 'b': -0.1909778265198026, 'c': 3.2373330032404954}}
Lap Number: 14
Lap Time: 90.384 seconds
Lap Time: 96.42223780729192 seconds


/tmp/ipykernel_20732/366242633.py:42: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  .filter(TyreRaceData.race_id.in_(last_20_race_ids))


## Sim Env

In [82]:
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import PPO

class F1RaceEnv(gym.Env):
	def __init__(self, total_laps, initial_tyre, tyre_params, baseline_lap, pit_time):
		super(F1RaceEnv, self).__init__()
		
		# Environment parameters
		self.total_laps = total_laps
		self.initial_tyre = initial_tyre
		self.tyre_params = tyre_params
		self.baseline_lap = baseline_lap
		self.pit_time = pit_time
		self.available_tyres = [1, 2, 3]  # Soft, Medium, Hard
		
		# Action space: [0 = no pit, 1 = pit soft, 2 = pit medium, 3 = pit hard]
		self.action_space = spaces.Discrete(4)
		
		# Observation space: [Lap Number, Tire Wear, Stint Laps, Pit Done, Remaining Race]
		self.observation_space = spaces.Box(
			low=0, 
			high=1, 
			shape=(5,), 
			dtype=np.float32
		)
		
		# Reset environment
		self.reset()

	def _get_lap_time(self, tyre, stint_laps, current_lap):
		params = self.tyre_params[tyre]
		max_fuel_kg = 110  # Maximum fuel load in kg
		fuel_effect_per_kg = 0.03  # Lap time increase per kg of fuel
		max_laps_race = self.total_laps  # Total laps in the race
		fuel_weight = max_fuel_kg - (current_lap - 1) * (max_fuel_kg / max_laps_race)
		fuel_correction = fuel_weight * fuel_effect_per_kg

		laptime = baseline_lap + fuel_correction + (params['a'] * stint_laps**2 + params['b'] * stint_laps + params['c'])


		return laptime

	def _get_state(self):
		state = [
			self.current_lap / self.total_laps,          # Normalized lap
			self.stint_laps / 20,                        # Normalized stint laps (max 20 laps/stint)
			float(self.pit_done),                        # Pit status
			self.current_tyre / len(self.available_tyres),  # Normalized tyre type
			(self.total_laps - self.current_lap) / self.total_laps  # Remaining race
		]
		return np.array(state, dtype=np.float32)

	def reset(self):
		self.current_lap = 1
		self.current_tyre = self.initial_tyre
		self.stint_laps = 1
		self.pit_done = False
		self.used_tyres = set([self.initial_tyre])  # Track used tyres
		return self._get_state()

	def step(self, action):
		done = False
		reward = 0
		info = {}

		# Calculate lap time for the current tyre
		lap_time = self._get_lap_time(self.current_tyre, self.stint_laps, self.current_lap)

		# Handle the action
		if action == 0:  # No pit
			self.stint_laps += 1
			reward -= lap_time

		elif action == 1:  # Pit and change to Soft tyre
			lap_time += self.pit_time
			reward -= lap_time
			self.current_tyre = 1  # Soft tyre
			if self.current_tyre not in self.used_tyres:
				reward += 50  # Bonus for new tyre compound
			self.used_tyres.add(self.current_tyre)  # Track the new tyre
			self.stint_laps = 1
			self.pit_done = True
			self.last_pit_lap = self.current_lap

		elif action == 2:  # Pit and change to Medium tyre
			lap_time += self.pit_time
			reward -= lap_time
			self.current_tyre = 2  # Medium tyre
			if self.current_tyre not in self.used_tyres:
				reward += 50  # Bonus for new tyre compound
			self.used_tyres.add(self.current_tyre)  # Track the new tyre
			self.stint_laps = 1
			self.pit_done = True
			self.last_pit_lap = self.current_lap

		elif action == 3:  # Pit and change to Hard tyre
			lap_time += self.pit_time
			reward -= lap_time
			self.current_tyre = 3  # Hard tyre
			if self.current_tyre not in self.used_tyres:
				reward += 50  # Bonus for new tyre compound
			self.used_tyres.add(self.current_tyre)  # Track the new tyre
			self.stint_laps = 1
			self.pit_done = True
			self.last_pit_lap = self.current_lap

		else:
			raise ValueError(f"Invalid action: {action}")

		# Move to the next lap
		self.current_lap += 1

		# Check race completion
		if self.current_lap > self.total_laps:
			done = True
			if not self.pit_done:
				reward -= 1000  # Penalty for missing pit stop
				info['reason'] = 'No pit stop'
			elif len(self.used_tyres) < 2:
				reward -= 1000  # Penalty for not using at least two tyre compounds
				info['reason'] = 'Less than two tyre compounds used'
			else:
				info['reason'] = 'Finished'

		# Return observation, reward, done, info
		return self._get_state(), reward, done, info

## RL Training


In [83]:
from stable_baselines3 import PPO

# Initialize environment
env = F1RaceEnv(total_laps, initial_tyre, tyre_params, baseline_lap, pit_time)

# Initialize PPO model
model = PPO("MlpPolicy", env, verbose=1, device="cpu")

# Train the model
model.learn(total_timesteps=99999)
model.save("f1_rl_model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/ben/Individual_Project/env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 57        |
|    ep_rew_mean     | -7.53e+03 |
| time/              |           |
|    fps             | 1344      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 57            |
|    ep_rew_mean          | -7.5e+03      |
| time/                   |               |
|    fps                  | 845           |
|    iterations           | 2             |
|    time_elapsed         | 4             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00040886542 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.39         |
|    explained_variance   | 0.0001

## Strat Optimisation


In [ ]:
import pandas as pd

def get_optimal_strategy(env, model):
	obs = env.reset()
	done = False
	strategy = []
	while not done:
		action, _ = model.predict(obs)
		obs, reward, done, _ = env.step(action)
		strategy.append({
			'lap': env.current_lap,
			'action': action
		})
	return strategy

# Load trained model
model = PPO.load("f1_rl_model")

# Generate optimal strategy
optimal_strategy = get_optimal_strategy(env, model)
# Convert strategy to a Pandas DataFrame
strategy_df = pd.DataFrame(optimal_strategy)

# Ensure all rows are displayed (no truncation)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

# Print the strategy in a readable format
print("Optimal Strategy:")
strategy_df


Optimal Strategy:


/home/ben/Individual_Project/env/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


,lap,action
0,2,3
1,3,0
2,4,0
3,5,0
4,6,0
5,7,0
6,8,0
7,9,0
8,10,0
9,11,0
